[REINVENT4 の環境](https://github.com/136s/REINVENT4/blob/6f10c90112b5fb3ccccc765c0fe1d085de3bfeb4/requirements-macOS-conda.yml) で作成した環境で実行

In [1]:
# P.33 リスト2.1: 2/smiles.py

from rdkit import Chem
from rdkit.Chem import Draw

mol = Chem.MolFromSmiles("CN1C=NC2=C1C(=O)N(C(=O)N2C)C")
Draw.MolToFile(mol, "caffeine.svg")

In [2]:
# P.43 リスト2.2: 2/test_selfies.py
# skip

In [3]:
# P.45 リスト2.4: 2/logp.py

from rdkit import Chem
from rdkit.Chem import Descriptors

mol = Chem.MolFromSmiles("CN1C=NC2=C1C(=O)N(C(=O)N2C)C")
logp = Descriptors.MolLogP(mol)
print("logp = {}".format(logp))

logp = -1.0293


# P.46 Morgan フィンガープリント

[文献 57](https://doi.org/10.1021/ci100050t) では、Daylight atomic invariants rule で原子をハッシュ化している。
この規則は以下の 6 つ性質を 32 ビット整数にハッシュ化している。

- 水素以外の近傍原子の数
- 価数から接続された水素の数を引いた値（言い換えれば、水素への結合を無視した総結合次数）
- 原子番号
- 原子質量
- 原子電荷
- 結合している水素の数（暗黙的および明示的の両方）

ECFP では「原子が少なくとも1つの環の一部であるかどうか」が追加されることがある。

In [4]:
# P.48 リスト2.5: 2/fingerprint.py

import torch
from rdkit import Chem
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprintAsBitVect

mol = Chem.MolFromSmiles("CN1C=NC2=C1C(=O)N(C(=O)N2C)C")
fp = GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2**11)
# rdkit 特有のデータ構造なので変換
fp_tensor = torch.tensor(fp)
fp_idx_tensor = torch.tensor(fp.GetOnBits())
print("fp_tensor = {}".format(fp_tensor))
print("shape = {}".format(fp_tensor.shape))  # fp.GetNumBits() と同じ
print("non-zero indices: {}".format(fp_idx_tensor))

fp_tensor = tensor([0, 0, 0,  ..., 0, 0, 0])
shape = torch.Size([2048])
non-zero indices: tensor([  33,  314,  378,  400,  463,  504,  564,  650,  771,  932,  935, 1024,
        1057, 1145, 1203, 1258, 1307, 1354, 1380, 1409, 1440, 1452, 1517, 1696,
        1873])


[23:35:18] DEPRECATION WARNING: please use MorganGenerator


In [5]:
# P.50 リスト2.7: 2/tanimoto.py

from rdkit import Chem
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprintAsBitVect
from rdkit import DataStructs

caffeine = Chem.MolFromSmiles("CN1C=NC2=C1C(=O)N(C(=O)N2C)C")
theophylline = Chem.MolFromSmiles("Cn1c2c(c(=O)n(c1=O)C)[nH]cn2")
fp_c = GetMorganFingerprintAsBitVect(caffeine, radius=2, nBits=2**11)
fp_t = GetMorganFingerprintAsBitVect(theophylline, radius=2, nBits=2**11)
print("Tanimoto similarity: {}".format(DataStructs.FingerprintSimilarity(fp_c, fp_t)))

Tanimoto similarity: 0.45714285714285713


[23:35:18] DEPRECATION WARNING: please use MorganGenerator
[23:35:18] DEPRECATION WARNING: please use MorganGenerator


In [6]:
# 谷本類似度の定義から計算

len(set(fp_c.GetOnBits()) & set(fp_t.GetOnBits())) / len(set(fp_c.GetOnBits()) | set(fp_t.GetOnBits()))

0.45714285714285713